#Paquetes necesarios

In [3]:
import cv2  
import math 

from ultralytics import YOLO

c:\Users\queol\anaconda3\envs\VC_P44\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modelos preentrenados, visualizando con las utilidades de ultralytics

In [6]:
# Carga del modelo
#model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolo11n-seg.pt') #Máscaras
model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()

100%|██████████| 5.97M/5.97M [00:00<00:00, 11.6MB/s]




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) c:\Users\queol\OneDrive - Universidad de Las Palmas de Gran Canaria\Desktop\VC_P4\otsedom.github.io-main\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 29.0ms
video 1/1 (frame 2/375) c:\Users\queol\OneDrive - Universidad de Las Palmas de Gran Canaria\Desktop\VC_P4\otsedom.github.io-main\VC\P4\TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 20.0ms
video 1/1 (frame 3/375) c:\Users\queol\OneDrive - Universidad de Las Palmas 

Desde cámara, detección con yolo11, modelo nano. Visualización propia con OpenCV

In [7]:
import torch
import os

# Comprobar si hay GPU disponible
device = 'cudaa' if torch.cuda.is_available() else 'cpu'
print(f"Usando: {device}")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Usando: cudaa


Seguimiento. Requiere instalar lap con pip install lap

In [30]:
import csv
from collections import defaultdict
import cv2
import math
import pytesseract
from collections import defaultdict
from PIL import Image

model = YOLO('yolo11n.pt') #Contenedores
model2 = YOLO('best.pt') #Contenedores
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Cambia esto según tu instalación
# Configuración de pytesseract
custom_config = r'--oem 3 --psm 8'
# Etiqueta de las distintas clases
classNames = ["person", "", "cars", "license_plate"]

# Variables adicionales
frame_count = 0
class_count = defaultdict(int)  # Contador para cada clase
output_video_path = 'output_video.avi'  # Ruta del video de salida
output_csv_path = 'detections.csv'  # Ruta del archivo CSV de salida

# Configuración para grabación de video
vid = cv2.VideoCapture("C0142.MP4")  # Ruta a tu video
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Inicialización de archivo CSV
with open(output_csv_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    # Cabecera del CSV
    writer.writerow([
        "fotograma", "tipo_objeto", "confianza", "identificador_tracking",
        "x1", "y1", "x2", "y2", "matrícula_en_su_caso", "mx1", "my1", "mx2", "my2", "texto_matricula"
    ])

track_history = defaultdict(lambda: [])

while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
    if not ret:
        break
    
    frame_count += 1  # Aumenta el contador de fotogramas
    bgr_img = cv2.cvtColor(img, cv2.COLOR_BGRBGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresholded_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    results = model.track(img, persist=True, classes = [0,2])
    results2 = model2.track(img, persist=True, classes = [0])
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            track_id = str(int(box.id[0].tolist())) if box.id is not None else ''
            confidence = round(float(box.conf[0]), 2)
            cls = int(box.cls[0])
            class_count[classNames[cls]] += 1  # Incrementa contador de la clase
            escala = int((cls / len(classNames)) * 255 * 3)
            R, G, B = (255, 255, escala - 255*2) if escala >= 255*2 else (255, escala - 255, 0) if escala >= 255 else (escala, 0, 0)

            # Dibuja y etiqueta
            cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
            cv2.putText(img, f"{track_id} {classNames[cls]}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)
            
            # Escribe al CSV (sin información de matrícula)
            with open(output_csv_path, mode='a', newline='') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow([frame_count, classNames[cls], confidence, track_id, x1, y1, x2, y2, '', '', '', '', ''])

    for r in results2:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            track_id = str(int(box.id[0].tolist())) if box.id is not None else ''
            confidence = round(float(box.conf[0]), 2)
            cls = 3
            class_count[classNames[cls]] += 1  # Incrementa contador de la clase "license_plate"
            escala = int((cls / len(classNames)) * 255 * 3)
            R, G, B = (255, 255, escala - 255*2) if escala >= 255*2 else (255, escala - 255, 0) if escala >= 255 else (escala, 0, 0)

            # OCR para la matrícula
            license_plate_img = thresholded_img[y1:y2, x1:x2]
            pil_img = Image.fromarray(license_plate_img)
            license_plate_text = pytesseract.image_to_string(pil_img, config=custom_config).strip()

            # Dibuja y etiqueta
            cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
            cv2.putText(img, license_plate_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(img, f"{track_id} {classNames[cls]}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

            # Escribe al CSV con la información de la matrícula
            with open(output_csv_path, mode='a', newline='') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow([
                    frame_count, classNames[cls], confidence, track_id, x1, y1, x2, y2,
                    license_plate_text, x1, y1, x2, y2, license_plate_text
                ])
    
    # Guarda el fotograma en el video de salida
    out_video.write(img)
    cv2.imshow('Vid', img)
    
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura y cierra los archivos de video y CSV
vid.release()
out_video.release()
cv2.destroyAllWindows()

# Imprime el total de detecciones por clase
print("Total de cada clase detectada:")
for cls, count in class_count.items():
    print(f"{cls}: {count}")



0: 384x640 4 cars, 46.0ms
Speed: 5.0ms preprocess, 46.0ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 256x416 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)

0: 384x640 4 cars, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 256x416 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)

0: 384x640 4 cars, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 256x416 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)

0: 384x640 4 cars, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 256x416 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per imag